In [1]:
# Install necessary libraries (run only once)
!pip install langchain faiss-cpu biopython sentence-transformers

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# -------------------------------
# Step 1: Load static medical data
# -------------------------------

def load_medical_knowledgebase():
    """
    Returns a list of brief medical summaries for common conditions.
    """
    return [
        "COVID-19: Management usually includes rest, fluids, antipyretics, and sometimes antivirals or oxygen therapy.",
        "Cancer: Treatment approaches include surgery, chemotherapy, radiation, and emerging immunotherapies.",
        "Diabetes: Controlled by monitoring glucose, using medications, maintaining a healthy diet, and staying active.",
        "Hypertension: Managed through blood pressure meds, lifestyle adjustments, and consistent monitoring.",
        "Asthma: Controlled with inhalers (bronchodilators/corticosteroids), avoiding triggers, and regular check-ups.",
        "Stroke: Recovery involves physical, speech, and occupational therapy for regaining daily functions.",
        "Alzheimer’s: Managed with cognitive enhancers, behavioral therapy, and structured daily routines.",
        "Migraine: Handled with painkillers, anti-nausea drugs, and lifestyle management to reduce triggers.",
        "Obesity: Tackled through a mix of diet changes, physical activity, counseling, and sometimes surgery.",
        "Heart Disease: Involves medication, lifestyle interventions, and surgical procedures like bypass.",
        "Kidney Issues: May require diet changes, medications, dialysis, or transplant in severe stages.",
        "Depression: Treatment includes antidepressants, therapy (CBT), and structured daily activity.",
        "Anxiety: Managed via therapy (like CBT), medication, mindfulness practices, and breathing techniques."
    ]

# -------------------------------
# Step 2: Create sentence embeddings
# -------------------------------

def embed_documents(texts):
    """
    Generates vector representations for a list of texts using Sentence Transformers.
    """
    encoder = SentenceTransformer('all-MiniLM-L6-v2')
    return encoder.encode(texts), encoder

# -------------------------------
# Step 3: Build FAISS similarity index
# -------------------------------

def setup_vector_index(embeddings):
    """
    Builds a FAISS index for fast similarity search using L2 distance.
    """
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index

# -------------------------------
# Step 4: Answer user questions
# -------------------------------

def find_best_match(query, faiss_index, source_texts, encoder, top_k=1):
    """
    Finds the most relevant document(s) for the given user query.
    """
    query_vec = encoder.encode([query])
    distances, indices = faiss_index.search(np.array(query_vec), k=top_k)
    return [(source_texts[i], distances[0][rank]) for rank, i in enumerate(indices[0])]

# -------------------------------
# Step 5: Start chatbot session
# -------------------------------

def run_medical_chatbot():
    """
    Runs an interactive terminal-based chatbot for answering medical queries.
    """
    knowledge_base = load_medical_knowledgebase()
    embeddings, encoder = embed_documents(knowledge_base)
    index = setup_vector_index(embeddings)

    print("🤖 Welcome to MedChat – your friendly medical info assistant!")
    print("💬 Ask me a health-related question (type 'exit' to quit).\n")

    while True:
        user_input = input("🩺 Your question: ")

        if user_input.lower().strip() == "exit":
            print("\n👋 Thank you for using MedChat. Stay healthy!")
            break

        results = find_best_match(user_input, index, knowledge_base, encoder, top_k=1)

        for i, (answer, score) in enumerate(results):
            print(f"\n📘 Match {i+1}: {answer} (relevance score: {round(score, 2)})\n")

# Run chatbot if this file is the entry point
if __name__ == "__main__":
    run_medical_chatbot()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🤖 Welcome to MedChat – your friendly medical info assistant!
💬 Ask me a health-related question (type 'exit' to quit).

🩺 Your question: What are the treatments for high blood pressure?

📘 Match 1: Hypertension: Managed through blood pressure meds, lifestyle adjustments, and consistent monitoring. (relevance score: 0.8100000023841858)

🩺 Your question: exit

👋 Thank you for using MedChat. Stay healthy!
